In [ ]:
import os
import cv2
import numpy as np
import matplotlib.pyplot as plt

import sys
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), "../..")))

from src.data_postprocessing import obtain_shoreline

In [ ]:
predictions_path = {
    "deeplabv3": os.path.abspath(os.path.join(os.getcwd(), "../../data/SCLabels_prediction_by_set_deeplab/")),
}

sation_names = ["agrelo", "arenaldentem", "cadiz", "cies", "samarador",]

# Path of the test images organized by station
test_path = "images/test/"
test_images_path = {
    "agrelo": [],
    "arenaldentem": [],
    "cadiz": [],
    "cies": [],
    "samarador": [],
}

for img_path in os.listdir(os.path.join(predictions_path["deeplabv3"], test_path)):
    if "agrelo" in img_path:
        test_images_path["agrelo"].append(img_path)
    elif "arenaldentem" in img_path:
        test_images_path["arenaldentem"].append(img_path)
    elif "cadiz" in img_path:
        test_images_path["cadiz"].append(img_path)
    elif "cies" in img_path:
        test_images_path["cies"].append(img_path)
    elif "samarador" in img_path:
        test_images_path["samarador"].append(img_path)

for station in sation_names:
    print(f"Station: {station}")
    print("length of test images path: ", len(test_images_path[station]))

In [ ]:
# Create a KDTree for the Ground Truth coordinates
from scipy.spatial import cKDTree

def compute_distance(mask_pred, mask_gt):
    coords_pred = np.column_stack(np.where(mask_pred == 1))
    coords_gt = np.column_stack(np.where(mask_gt == 1))

    # Distance from predicted to GT
    # Create a KDTree for the Ground Truth coordinates
    tree_gt = cKDTree(coords_gt)

    # Find the nearest neighbors in the Ground Truth for each coordinate in the predicted mask
    dists_pred_to_gt, _ = tree_gt.query(coords_pred)

    # Distance from GT to predicted
    # Create a KDTree for the predicted coordinates
    tree_pred = cKDTree(coords_pred)
    # Find the nearest neighbors in the predicted for each coordinate in the GT mask
    dists_gt_to_pred, _ = tree_pred.query(coords_gt)

    return dists_pred_to_gt, dists_gt_to_pred

def apply_masks(image, mask, predicted_mask, shoreline_pixel_mask, shoreline_pixel_predicted_mask):
    # Copy original image
    overlay = image.copy()

    alpha = 0.6

    green = np.array([0, 255, 0], dtype=np.uint8)     # Shoreline
    red = np.array([255, 0, 0], dtype=np.uint8)       # Predicted shoreline
    yellow = np.array([255, 255, 0], dtype=np.uint8)  # Combined shoreline

    # Create masks for shoreline pixels
    only_shoreline = (mask == shoreline_pixel_mask) & (predicted_mask != shoreline_pixel_predicted_mask)
    only_predicted = (mask != shoreline_pixel_mask) & (predicted_mask == shoreline_pixel_predicted_mask)
    both = (mask == shoreline_pixel_mask) & (predicted_mask == shoreline_pixel_predicted_mask)

    # Apply colors to the overlay
    overlay[only_shoreline] = (
        alpha * green + (1 - alpha) * overlay[only_shoreline]
    ).astype(np.uint8)

    overlay[only_predicted] = (
        alpha * red + (1 - alpha) * overlay[only_predicted]
    ).astype(np.uint8)

    overlay[both] = (
        alpha * yellow + (1 - alpha) * overlay[both]
    ).astype(np.uint8)

    return overlay

def get_image(predictions_path, test_images_path, station, index, shoreline_pixel_mask, shoreline_pixel_predicted_mask):
    path_img = os.path.join(predictions_path, "images/test",test_images_path[station][index])
    print(f"Reading image from: {path_img}")
    img = cv2.imread(path_img)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

    # Read mask
    path_mask = os.path.join(predictions_path, "masks/test",test_images_path[station][index].replace("image.jpg", "mask.png"))
    mask = cv2.imread(path_mask, cv2.IMREAD_GRAYSCALE)

    # Read predicted mask
    path_predicted_mask = os.path.join(predictions_path, "predicted_mask/test",test_images_path[station][index].replace("image.jpg", "mask.png"))
    
    predicted_mask = obtain_shoreline.transform_mask_to_shoreline(pred_path = path_predicted_mask, landward=1, seaward=2)
    mask[mask == 3] = 1  # Convert shoreline class to landward class
    mask = obtain_shoreline.transform_mask_to_shoreline_from_img(pred = mask, landward=1, seaward=2)
    new_mask = np.zeros_like(mask)
    new_mask[mask == 1] = shoreline_pixel_mask  # Landward
    mask = new_mask

    combined_image = apply_masks(img, mask, predicted_mask, shoreline_pixel_mask=shoreline_pixel_mask, shoreline_pixel_predicted_mask=shoreline_pixel_predicted_mask)

    return combined_image

In [ ]:
# Create directories for saving predictions
path = os.path.abspath(os.path.join(os.getcwd(), "../../predictions/station"))

os.makedirs(path, exist_ok=True)
for station_name in sation_names:
    os.makedirs(os.path.join(path, station_name), exist_ok=True)

In [ ]:
import numpy as np
import cv2

for station_name in sation_names:
    print(f"Station: {station_name}")
    for index in range(len(test_images_path[station_name])):
        img = get_image(predictions_path["deeplabv3"], test_images_path, station_name, index, shoreline_pixel_mask = 1, shoreline_pixel_predicted_mask = 1)

        prediction_path_name = os.path.join(path, station_name, test_images_path[station_name][index].replace("image.jpg", "prediction.png"))
        cv2.imwrite(prediction_path_name, cv2.cvtColor(img, cv2.COLOR_RGB2BGR))


# Create the split folder

In [54]:
# Create directories for saving predictions
path = os.path.abspath(os.path.join(os.getcwd(), "../../predictions/splits"))
os.makedirs(path, exist_ok=True)

In [ ]:
splits = {
    "train": "images/train/",
    "val": "images/validation/",
    "test": "images/test/",
}

for split_name, relative_dir in splits.items():
    print(f"Creating directory for split: {split_name}")

    full_split_dir = os.path.join(predictions_path["deeplabv3"], relative_dir)
    
    entries = []
    for filename in os.listdir(full_split_dir):
        entries.append(filename)

    # Save the entries to a text file
    split_file_path = os.path.join(path, f"{split_name}.txt")
    with open(split_file_path, 'w') as f:
        for entry in entries:
            f.write(f"{entry}\n")
    

Creating directory for split: train
Creating directory for split: val
Creating directory for split: test
